In [1]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

trials = 5

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
edges_to_add = None

In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model_save_path = "../models/cora_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges_full_run.pth"
# list_save_path = "../attacks/cora_gcn_edges.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [7]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [8]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [9]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.20)

In [ ]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_greedy(data, train, model, 0.0, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6076424717903137
Surrogate acc on unlabled data: 0.805
Attack loss: 0.2829137444496155
Surrogate loss on unlabled data: 0.7039027810096741
Surrogate acc on unlabled data: 0.781
Attack loss: 0.4154606759548187
Surrogate loss on unlabled data: 0.9057208299636841
Surrogate acc on unlabled data: 0.736
Attack loss: 0.6127167344093323
Surrogate loss on unlabled data: 0.8646247982978821
Surrogate acc on unlabled data: 0.725
Attack loss: 0.6141096353530884
Surrogate loss on unlabled data: 1.0203737020492554
Surrogate acc on unlabled data: 0.682
Attack loss: 0.7593544721603394
Surrogate loss on unlabled data: 1.1040681600570679
Surrogate acc on unlabled data: 0.645
Attack loss: 0.867948591709137
Surrogate loss on unlabled data: 1.1518312692642212
Surrogate acc on unlabled data: 0.642
Attack loss: 0.9360206723213196
Surrogate loss on unlabled data: 1.2556995153427124
Surrogate acc on unlabled data: 0.635
Attack loss: 1.0342254638671875
Surrogate loss on unlable

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5966231226921082
Surrogate acc on unlabled data: 0.808
Attack loss: 0.28638315200805664
Surrogate loss on unlabled data: 0.7257119417190552
Surrogate acc on unlabled data: 0.78
Attack loss: 0.4335567057132721
Surrogate loss on unlabled data: 0.8151730298995972
Surrogate acc on unlabled data: 0.751
Attack loss: 0.519555926322937
Surrogate loss on unlabled data: 0.9013957381248474
Surrogate acc on unlabled data: 0.741
Attack loss: 0.630111813545227
Surrogate loss on unlabled data: 0.987515389919281
Surrogate acc on unlabled data: 0.714
Attack loss: 0.7207518815994263
Surrogate loss on unlabled data: 1.046435832977295
Surrogate acc on unlabled data: 0.6910000000000001
Attack loss: 0.8036395907402039
Surrogate loss on unlabled data: 1.1538233757019043
Surrogate acc on unlabled data: 0.659
Attack loss: 0.9270074367523193
Surrogate loss on unlabled data: 1.2712140083312988
Surrogate acc on unlabled data: 0.631
Attack loss: 1.0364372730255127
Surrogate loss 

In [ ]:
edges_to_add

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

# regular, secondary attack, not reversed

In [ ]:
# split_dic_acc

In [ ]:
splits = [0.0, 0.5]

split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [ ]:
# data

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_loss, split_dic_acc, itrs_one, Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"cora"}_{"constant"}_{"attacked"}_results.pkl')

In [ ]:
split_dic_loss

In [ ]:
# plot_results({0: split_dic_acc[0.0][0], 0.5: split_dic_acc[0.5][0]}, ptb_rate, "Greedy", "", "constant", "accuracy", "regular")

In [ ]:
# plot_results({0: split_dic_loss[0.0][0], 0.5: split_dic_loss[0.5][0]}, ptb_rate, "Greedy", "", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [ ]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_decreasing_loss

In [ ]:
record1 = Recording(split_dic_decreasing_loss, split_dic_decreasing_acc, itrs_two, Model.GCN, Dataset.CORA, AcceptFn.INCREASING, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"cora"}_{"increasing"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results({0: split_dic_decreasing_acc[0.0][0], 0.5: split_dic_decreasing_acc[0.5][0]}, ptb_rate, "Greedy", "", "increasing", "accuracy", "regular")

In [ ]:
# plot_results({0: split_dic_decreasing_loss[0.0][0], 0.5: split_dic_decreasing_loss[0.5][0]}, ptb_rate, "Greedy", "", "increasing", "loss", "regular")